In [2]:

#input: the list of edges
#output: incidence matrix of the hypergraph
def edges_to_matrix(E):
    H = IncidenceStructure(E)
    M = H.incidence_matrix()
    return M

#input: the incidence matrix
#output: the list of edges
def matrix_to_edges(M):
    H = IncidenceStructure(M)
    E=[]
    for element in H.blocks():
        E.append(tuple(element))
    return E


# input: hypergraph incidence matrix M, vertices X edges
# output: list of all rootings as a list of nonzero (row,col) pairs of M
def rootings(M):
    R = M.nrows()
    if R == 0 or M.ncols() == 0:
        return [[]]
    rootinglist = []
    for rownum in range(R):
        if M[rownum][0] == 0:
            rownum += 1
        else:
            tempmx = M.submatrix(0,1)
            for colnum in range(1,M.ncols()):
                if M[:,colnum] == M[:,0]:
                    for j in range(rownum):
                        tempmx[j,colnum-1] = 0
            templist = rootings(tempmx)
            rootinglist += [[rownum]+L for L in templist]
    return rootinglist


# input: hypergraph incidence matrix M, vertices X edges
# output: list of all digraphs of Euler rootings
def euler_rootings_digraph(M):
    rootinglist = rootings(M)
    R = M.nrows()
    returnlist = []
    edges=matrix_to_edges(M)
    dit=dict([(a,0) for a in edges])
    for L in rootinglist:
        A = zero_matrix(R)
        c=1
        for rownum in range(R):
            dicct=copy(dit)
            for j in range(len(L)):
                if M[rownum,j] != 0:
                    if rownum != L[j]:
                        A[L[j],rownum] += 1
                    else:
                        dicct[edges[j]]+=1
            c*=multinomial(dicct.values())
        D = DiGraph(A)
        if D.is_eulerian():
#             show(D)
            returnlist.append((D,c))
    return returnlist

#in: Eulerian digraph D
#out: number of spanning rooted subtrees of D
def tau(D):
    L = D.laplacian_matrix()
    L1 = L.submatrix(1,1)
    tau = L1.determinant()
    return tau


#in: distinguishable Eulerian digraph D
#out: number of distinguishable Euler tours of D
def count_euler_tours(D):
    c=tau(D)
    for v in D.vertices():
        c *= factorial(D.out_degree(v)-1)
    return(c)

#input: list of edges of a k-uniform hypergraph
#output: Associated coefficient of H
def as_co(E):
    M=edges_to_matrix(E)
    s=0
    for pair in euler_rootings_digraph(M):
        D=pair[0] 
        c=1
        for v in D.vertices():
            c*=D.out_degree(v)
        s+=pair[1]*tau(D)/c
        # show(D)
    return s

#Tight Path of length 2
E= [[0,1,2],[0,1,2],[0,1,2],[1,2,3],[1,2,3],[1,2,3]]
print(as_co(E))

9/8


/tmp/ipykernel_779/326161395.py:90: DeprecationWarning: parameter 'sort' will be set to False by default in the future
See http://trac.sagemath.org/22349 for details.
  for v in D.vertices():
